In [14]:
# print("Okay!")

In [2]:
%pwd

'd:\\PERSONAL PROJECTS\\MediBot\\research'

In [3]:
import os
os.chdir("../")
%pwd

'd:\\PERSONAL PROJECTS\\MediBot'

In [72]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [73]:
#Extract Data from the MedBook PDF File
def loadpdf_file(data):
    loader= DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    docs= loader.load()
    
    return docs

In [74]:
extracted_data= loadpdf_file(data='Data/')

In [8]:
# extracted_data

In [75]:
#Split the Data into Text Chunks
def txt_split(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks


In [76]:
text_chunks= txt_split(extracted_data)
print("Lenght of Text Chunks: ", len(text_chunks))

Lenght of Text Chunks:  7023


In [12]:
# text_chunks

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [77]:
#Download the Embeddings from Hugging Face
def download_hug_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [78]:
embeddings = download_hug_face_embeddings()

In [79]:
query_res= embeddings.embed_query("Hello World")
print("Length: ", len(query_res))
# print("Vector: ", query_res)

Length:  384


In [80]:
from dotenv import load_dotenv
load_dotenv()

True

In [81]:
PINECONE_APIKEY= os.environ.get("PINECONE_APIKEY")

In [ ]:
# import pinecone

from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_APIKEY)

# pinecone.init(api_key=,environment='us-west1-gcp')

index_name="medicalchatbot"
dimension= 384

if index_name not in Pinecone.list_indexes():
    Pinecone.create_index(name=index_name, dimension=dimension)
    
index= Pinecone.Index(index_name)
print(f"Connected to the index:{index_name}")

In [66]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_APIKEY 

In [ ]:
#Embed each chunk and upsert the embeddings into your PineCone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    docs=text_chunks,
    index_name=index_name,
    embeddings=embeddings,
    )